In [1]:
import os
os.chdir('..')

import pandas as pd
import numpy as np
import matplotlib.pylab as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)



/var/folders/m5/9j_7j_8j5m154hyxw8j2346h0000gn/T/ipykernel_53434/1197053385.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
data = pd.read_pickle('data/data.pkl')
target_col = 'y'
target = data[target_col]
data.head(16)

,askRate0,askRate1,askRate2,askRate3,askRate4,askRate5,askRate6,askRate7,askRate8,askRate9,askRate10,askRate11,askRate12,askRate13,askRate14,askSize0,askSize1,askSize2,askSize3,askSize4,askSize5,askSize6,askSize7,askSize8,askSize9,askSize10,askSize11,askSize12,askSize13,askSize14,bidRate0,bidRate1,bidRate2,bidRate3,bidRate4,bidRate5,bidRate6,bidRate7,bidRate8,bidRate9,bidRate10,bidRate11,bidRate12,bidRate13,bidRate14,bidSize0,bidSize1,bidSize2,bidSize3,bidSize4,bidSize5,bidSize6,bidSize7,bidSize8,bidSize9,bidSize10,bidSize11,bidSize12,bidSize13,bidSize14,y
0,1619.5,1620.0,1621.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,0,0,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5
1,1619.5,1620.0,1621.0,1621.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,5,0,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5
2,1619.5,1620.0,1621.0,1621.5,1622.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,5,2,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5
3,1619.5,1620.0,1621.0,1621.5,1622.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,5,22,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5
4,1619.5,1620.0,1621.0,1621.5,1622.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,5,32,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5
5,1619.5,1620.0,1621.0,1621.5,1622.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,5,152,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5
6,1619.5,1620.0,1621.0,1621.5,1622.0,1623.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,5,152,10,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5
7,1619.5,1620.0,1621.0,1621.5,1622.0,1623.0,1623.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,5,152,10,2,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5
8,1619.5,1620.0,1621.0,1621.5,1622.0,1623.0,1623.5,1624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,5,152,10,2,8,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5
9,1619.5,1620.0,1621.0,1621.5,1622.0,1623.0,1623.5,1624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,24,5,152,10,2,18,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13,-0.5


In [3]:
ask_rate_cols = [f'askRate{idx}' for idx in range(15)]
ask_size_cols = [f'askSize{idx}' for idx in range(15)]
bid_rate_cols = [f'bidRate{idx}' for idx in range(15)]
bid_size_cols = [f'bidSize{idx}' for idx in range(15)]

ask_rate = [data.loc[:, col].copy() for col in ask_rate_cols]
bid_rate = [data.loc[:, col].copy() for col in bid_rate_cols]
ask_size = [data.loc[:, col].copy() for col in ask_size_cols]
bid_size = [data.loc[:, col].copy() for col in bid_size_cols]

def calc_wap(order: int):
    return (bid_rate[order] * ask_size[order] + ask_rate[order] * bid_size[order]) / (ask_size[order] + bid_size[order])
def calc_wapcos(order: int):
    return (bid_rate[order] * bid_size[order] + ask_rate[order] * ask_size[order]) / (ask_size[order] + bid_size[order])

In [5]:
features_list = []
for n_per_row in (5, 10, 15, 25, 50, 100):
    current_features = pd.read_pickle(f'../artefacts/features_{n_per_row}.pkl')
    current_features.drop([f'ask_flatten_len_{n_per_row}', f'bid_flatten_len_{n_per_row}'], axis=1, inplace=True)
    features_list.append(current_features)
features = pd.concat(features_list, axis=1)

features['bid_ask_spread'] = ask_rate[0] / bid_rate[0] - 1
features['wap0'] = calc_wap(0)
features['wap1'] = calc_wap(1).astype(np.float32)
features['wapcos0'] = calc_wapcos(0).astype(np.float32)
features['wapcos1'] = calc_wapcos(1).astype(np.float32)

features = pd.concat((features, data.drop(['y'], 1)), axis=1)
features.head()

/var/folders/m5/9j_7j_8j5m154hyxw8j2346h0000gn/T/ipykernel_53434/4023804390.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features = pd.concat((features, data.drop(['y'], 1)), axis=1)


,ask_flatten_mean_5,ask_flatten_median_5,ask_flatten_std_5,ask_flatten_iqr_5,ask_flatten_skew_5,ask_flatten_kurtosis_5,bid_flatten_mean_5,bid_flatten_median_5,bid_flatten_iqr_5,bid_flatten_skew_5,bid_flatten_kurtosis_5,flatten_spread_5_mean,flatten_spread_5_median,wap_flatten_5,ask_flatten_mean_10,ask_flatten_median_10,ask_flatten_std_10,ask_flatten_iqr_10,ask_flatten_skew_10,ask_flatten_kurtosis_10,bid_flatten_mean_10,bid_flatten_median_10,bid_flatten_iqr_10,bid_flatten_skew_10,bid_flatten_kurtosis_10,flatten_spread_10_mean,flatten_spread_10_median,wap_flatten_10,ask_flatten_mean_15,ask_flatten_median_15,ask_flatten_std_15,ask_flatten_iqr_15,ask_flatten_skew_15,ask_flatten_kurtosis_15,bid_flatten_mean_15,bid_flatten_median_15,bid_flatten_iqr_15,bid_flatten_skew_15,bid_flatten_kurtosis_15,flatten_spread_15_mean,flatten_spread_15_median,wap_flatten_15,ask_flatten_mean_25,ask_flatten_median_25,ask_flatten_std_25,ask_flatten_iqr_25,ask_flatten_skew_25,ask_flatten_kurtosis_25,bid_flatten_mean_25,bid_flatten_median_25,...,askRate10,askRate11,askRate12,askRate13,askRate14,askSize0,askSize1,askSize2,askSize3,askSize4,askSize5,askSize6,askSize7,askSize8,askSize9,askSize10,askSize11,askSize12,askSize13,askSize14,bidRate0,bidRate1,bidRate2,bidRate3,bidRate4,bidRate5,bidRate6,bidRate7,bidRate8,bidRate9,bidRate10,bidRate11,bidRate12,bidRate13,bidRate14,bidSize0,bidSize1,bidSize2,bidSize3,bidSize4,bidSize5,bidSize6,bidSize7,bidSize8,bidSize9,bidSize10,bidSize11,bidSize12,bidSize13,bidSize14
0,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.003096,1617.45,1619.95,1620.0,0.15,0.0,-2.666667,5.111111,1614.7,1615.0,0.75,-0.872872,-1.238095,0.003251,0.003096,1617.325,1620.233333,1620.0,0.478423,0.5,0.779935,-0.835753,1614.466667,1614.0,1.0,0.133631,-1.982143,0.003572,0.003717,1617.35,1620.54,1621.0,0.527636,1.0,-0.364018,-1.633918,1613.68,1614.0,...,NaN,NaN,NaN,NaN,NaN,1,10,24,0,0,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13
1,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.003096,1617.45,1619.95,1620.0,0.15,0.0,-2.666667,5.111111,1614.7,1615.0,0.75,-0.872872,-1.238095,0.003251,0.003096,1617.325,1620.233333,1620.0,0.478423,0.5,0.779935,-0.835753,1614.466667,1614.0,1.0,0.133631,-1.982143,0.003572,0.003717,1617.35,1620.54,1621.0,0.527636,1.0,-0.364018,-1.633918,1613.68,1614.0,...,NaN,NaN,NaN,NaN,NaN,1,10,24,5,0,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13
2,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.003096,1617.45,1619.95,1620.0,0.15,0.0,-2.666667,5.111111,1614.7,1615.0,0.75,-0.872872,-1.238095,0.003251,0.003096,1617.325,1620.233333,1620.0,0.478423,0.5,0.779935,-0.835753,1614.466667,1614.0,1.0,0.133631,-1.982143,0.003572,0.003717,1617.35,1620.54,1621.0,0.527636,1.0,-0.364018,-1.633918,1613.68,1614.0,...,NaN,NaN,NaN,NaN,NaN,1,10,24,5,2,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13
3,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.003096,1617.45,1619.95,1620.0,0.15,0.0,-2.666667,5.111111,1614.7,1615.0,0.75,-0.872872,-1.238095,0.003251,0.003096,1617.325,1620.233333,1620.0,0.478423,0.5,0.779935,-0.835753,1614.466667,1614.0,1.0,0.133631,-1.982143,0.003572,0.003717,1617.35,1620.54,1621.0,0.527636,1.0,-0.364018,-1.633918,1613.68,1614.0,...,NaN,NaN,NaN,NaN,NaN,1,10,24,5,22,0,0,0,0,0,0,0,0,0,0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13
4,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.003096,1617.45,1619.95,1620.0,0.15,0.0,-2.666667,5.111111,1614.7,1615.0,0.75,-0.872872,-1.238095,0.00325

In [8]:
# rolling_features = pd.read_pickle('rolling_features.pkl')
# rolling_features.shape

In [7]:
rolling_features.columns

NameError: name 'rolling_features' is not defined

In [5]:
# from tqdm import tqdm
# import scipy.stats
# for col in tqdm(selected_cols):
#     for window_size in [10, 100, 1000]:
#         rolling = features[col].rolling(window_size)
#         prefix = f'{col}_rolling_{window_size}'
#         features[f'{prefix}_mean_{window_size}'] = rolling.parallel_apply(np.mean)
#         features[f'{prefix}_median_{window_size}'] = rolling.parallel_apply(np.median)
#         features[f'{prefix}_std_{window_size}'] = rolling.parallel_apply(np.std)
#         features[f'{prefix}_iqr_{window_size}'] = rolling.parallel_apply(scipy.stats.iqr)
#         features[f'{prefix}_skew_{window_size}'] = rolling.parallel_apply(scipy.stats.skew)
#         features[f'{prefix}_kurtosis_{window_size}'] = rolling.parallel_apply(scipy.stats.kurtosis)

In [6]:
# features_norm = (features - features.mean()) / features.std()
# features_norm.head()

In [8]:
# rolling_cols = [col for col in features.columns if 'rolling' in col]
# features[rolling_cols].to_pickle('rolling_features.pkl')

In [13]:
def sign(x):
    if x == 0:
        return x
    return x / abs(x)

data.y.apply(sign).value_counts()

 1.0    1364120
-1.0    1362455
 0.0     771091
Name: y, dtype: int64

In [143]:
features = pd.concat((features, rolling_features), axis=1)
features.shape

LICENSE               data/                 rolling_features.pkl
LightGBM/             logging/              xtx/
README.md             notebooks/
artefacts/            requirements.txt


In [14]:
class Splitter():
    def __init__(self, test_size):
        self.test_size = test_size
    
    def __call__(self, data, target):
        idx = data.notnull().all(axis=1)
        selected_data = data.loc[idx, :].copy()
        selected_target = target.loc[idx].copy()

        n_train = int((1 - self.test_size) * selected_data.shape[0])
        return selected_data[:n_train], selected_data[n_train:], selected_target[:n_train], selected_target[n_train:]
# data.fillna(0, inplace=True)
train_data, test_data, train_target, test_target = Splitter(0.25)(features, data.y)

In [15]:
train_data_norm = (train_data - train_data.mean()) / train_data.std()
test_data_norm = (test_data - train_data.mean()) / train_data.std()


In [16]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=100)
model.fit(train_data_norm, train_target)
predicted = model.predict(test_data_norm)
corr_score = np.corrcoef(predicted, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

Correlation: 0.145356


In [18]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(n_jobs=-1)
model.fit(train_data_norm, train_target.apply(sign))
print('fit performed')
predicted = model.predict_proba(test_data_norm)
print(predicted.shape)
# corr_score = np.corrcoef(predicted, test_target)[0,1]
# print(f'Correlation: {corr_score:.6f}')

/Users/sneddy/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


fit performed
(874273, 3)


In [37]:
sqrt_probas = predicted**0.5
score = (predicted[:, 2] - predicted[:, 0]) * (1 - predicted[:, 1])
corr_score = np.corrcoef(score, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

Correlation: 0.146062


In [8]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.01)
model.fit(train_data_norm, train_target)
predicted = model.predict(test_data_norm)
corr_score = np.corrcoef(predicted, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

Correlation: 0.151077


In [12]:
selected_cols_idx = np.where(model.coef_ != 0)[0]
selected_cols = train_data_norm.columns[selected_cols_idx].tolist()
print(f'Selected by Lasso: {selected_cols_idx.shape[0]} from {model.coef_.shape[0]}')
print(selected_cols)

Selected by Lasso: 18 from 149
['ask_flatten_std_5', 'ask_flatten_skew_5', 'ask_flatten_kurtosis_5', 'bid_flatten_iqr_5', 'bid_flatten_skew_5', 'bid_flatten_kurtosis_5', 'flatten_spread_5_mean', 'ask_flatten_std_10', 'bid_flatten_iqr_10', 'ask_flatten_std_15', 'bid_flatten_iqr_15', 'ask_flatten_std_25', 'bid_flatten_iqr_25', 'bid_flatten_kurtosis_25', 'bid_flatten_iqr_50', 'ask_flatten_std_100', 'bid_flatten_iqr_100', 'askSize1']


In [17]:
from lightgbm import LGBMRegressor
# model = LGBMRegressor(n_jobs=-1, num_leaves=32, learning_rate=0.01, n_estimators=300, reg_lambda=1, colsample_bytree=0.3, subsample=0.2)
model = LGBMRegressor(n_jobs=-1, num_leaves=13, learning_rate=0.01, n_estimators=300, reg_lambda=1, colsample_bytree=0.7, subsample=0.7)
model.fit(train_data[selected_cols], train_target)
predicted = model.predict(test_data[selected_cols])
corr_score = np.corrcoef(predicted, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030748 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2392
[LightGBM] [Info] Number of data points in the train set: 2622818, number of used features: 18
[LightGBM] [Info] Start training from score 0.001244
Correlation: 0.149795


In [41]:
# train_data = train_data[selected_cols]
# test_data = test_data[selected_cols]
train_data['ask_flatten_mean_5_rolling_std'] = train_data['ask_flatten_mean_5'].rolling(100).parallel_apply(np.std).fillna(0)
test_data['ask_flatten_mean_5_rolling_std'] = test_data['ask_flatten_mean_5'].rolling(100).parallel_apply(np.std).fillna(0)

In [39]:
train_data['ask_flatten_mean_5']

12         1619.9
13         1619.9
14         1619.9
15         1619.9
16         1619.9
            ...  
2623207    1645.0
2623208    1645.0
2623209    1645.0
2623210    1645.0
2623211    1645.0
Name: ask_flatten_mean_5, Length: 2622818, dtype: float64

In [42]:
from lightgbm import LGBMRegressor
extended_cols = selected_cols + ['ask_flatten_mean_5_rolling_std']
# model = LGBMRegressor(n_jobs=-1, num_leaves=32, learning_rate=0.01, n_estimators=300, reg_lambda=1, colsample_bytree=0.3, subsample=0.2)
model = LGBMRegressor(n_jobs=-1, num_leaves=13, learning_rate=0.01, n_estimators=300, reg_lambda=1, colsample_bytree=0.7, subsample=0.7)
model.fit(train_data[extended_cols], train_target)
predicted = model.predict(test_data[extended_cols])
corr_score = np.corrcoef(predicted, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029547 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2650
[LightGBM] [Info] Number of data points in the train set: 2622818, number of used features: 19
[LightGBM] [Info] Start training from score 0.001244
Correlation: 0.149544


In [32]:
train_data['ask_flatten_mean_5_rolling_std'].fillna(0, inplace=True)

In [14]:
train_data.head()

,ask_flatten_mean_5,ask_flatten_median_5,ask_flatten_std_5,ask_flatten_iqr_5,ask_flatten_skew_5,ask_flatten_kurtosis_5,bid_flatten_mean_5,bid_flatten_median_5,bid_flatten_iqr_5,bid_flatten_skew_5,bid_flatten_kurtosis_5,flatten_spread_5_mean,flatten_spread_5_median,wap_flatten_5,ask_flatten_mean_10,ask_flatten_median_10,ask_flatten_std_10,ask_flatten_iqr_10,ask_flatten_skew_10,ask_flatten_kurtosis_10,bid_flatten_mean_10,bid_flatten_median_10,bid_flatten_iqr_10,bid_flatten_skew_10,bid_flatten_kurtosis_10,flatten_spread_10_mean,flatten_spread_10_median,wap_flatten_10,ask_flatten_mean_15,ask_flatten_median_15,ask_flatten_std_15,ask_flatten_iqr_15,ask_flatten_skew_15,ask_flatten_kurtosis_15,bid_flatten_mean_15,bid_flatten_median_15,bid_flatten_iqr_15,bid_flatten_skew_15,bid_flatten_kurtosis_15,flatten_spread_15_mean,flatten_spread_15_median,wap_flatten_15,ask_flatten_mean_25,ask_flatten_median_25,ask_flatten_std_25,ask_flatten_iqr_25,ask_flatten_skew_25,ask_flatten_kurtosis_25,bid_flatten_mean_25,bid_flatten_median_25,...,askRate10,askRate11,askRate12,askRate13,askRate14,askSize0,askSize1,askSize2,askSize3,askSize4,askSize5,askSize6,askSize7,askSize8,askSize9,askSize10,askSize11,askSize12,askSize13,askSize14,bidRate0,bidRate1,bidRate2,bidRate3,bidRate4,bidRate5,bidRate6,bidRate7,bidRate8,bidRate9,bidRate10,bidRate11,bidRate12,bidRate13,bidRate14,bidSize0,bidSize1,bidSize2,bidSize3,bidSize4,bidSize5,bidSize6,bidSize7,bidSize8,bidSize9,bidSize10,bidSize11,bidSize12,bidSize13,bidSize14
12,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.003096,1617.45,1619.95,1620.0,0.15,0.0,-2.666667,5.111111,1614.7,1615.0,0.75,-0.872872,-1.238095,0.003251,0.003096,1617.325,1620.233333,1620.0,0.478423,0.5,0.779935,-0.835753,1614.466667,1614.0,1.0,0.133631,-1.982143,0.003572,0.003717,1617.35,1620.54,1621.0,0.527636,1.0,-0.364018,-1.633918,1613.68,1614.0,...,1627.0,1628.0,1628.5,1629.0,1630.0,1,10,24,5,152,10,2,24,82,10,11,10,1,12,3,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,20,3,20,27,11,14,35,10,1,10,13
13,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.003096,1617.45,1619.95,1620.0,0.15,0.0,-2.666667,5.111111,1614.7,1615.0,0.75,-0.872872,-1.238095,0.003251,0.003096,1617.325,1620.233333,1620.0,0.478423,0.5,0.779935,-0.835753,1614.466667,1614.0,1.0,0.133631,-1.982143,0.003572,0.003717,1617.35,1620.54,1621.0,0.527636,1.0,-0.364018,-1.633918,1613.68,1614.0,...,1627.0,1628.0,1628.5,1629.0,1630.0,1,10,24,5,152,10,2,24,82,10,11,10,1,12,3,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,21,3,20,27,11,14,35,10,1,10,13
14,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.003096,1617.45,1619.95,1620.0,0.15,0.0,-2.666667,5.111111,1614.7,1615.0,0.75,-0.872872,-1.238095,0.003251,0.003096,1617.325,1620.233333,1620.0,0.478423,0.5,0.779935,-0.835753,1614.466667,1614.0,1.0,0.133631,-1.982143,0.003572,0.003717,1617.35,1620.54,1621.0,0.527636,1.0,-0.364018,-1.633918,1613.68,1614.0,...,1627.0,1628.0,1628.5,1629.0,1630.0,1,10,24,6,152,10,2,24,82,10,11,10,1,12,3,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,1600.0,7,10,1,10,21,3,20,27,11,14,35,10,1,10,13
15,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.003096,1617.45,1619.95,1620.0,0.15,0.0,-2.666667,5.111111,1614.7,1615.0,0.75,-0.872872,-1.238095,0.003251,0.003096,1617.325,1620.233333,1620.0,0.478423,0.5,0.779935,-0.835753,1614.466667,1614.0,1.0,0.133631,-1.982143,0.003572,0.003717,1617.35,1620.54,1621.0,0.527636,1.0,-0.364018,-1.633918,1613.68,1614.0,...,1627.0,1628.0,1628.5,1629.0,1630.0,1,10,24,6,152,10,2,24,82,10,11,10,1,12,3,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.5,1606.0,1605.0,1604.0,1603.0,1602.0,1601.5,1601.0,7,10,1,10,21,3,20,2,27,11,14,35,10,1,10
16,1619.9,1620.0,0.2,0.0,-1.5,0.25,1615.0,1615.0,0.0,0.0,-3.0,0.003034,0.00309

In [10]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=100)
model.fit(train_data_norm, train_target)
predicted = model.predict(test_data_norm)
corr_score = np.corrcoef(predicted, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

Correlation: 0.140007


In [11]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.01)
model.fit(train_data_norm, train_target)
predicted = model.predict(test_data_norm)
corr_score = np.corrcoef(predicted, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

Correlation: 0.151077


In [12]:
selected_cols_idx = np.where(model.coef_ != 0)[0]
selected_cols = train_data_norm.columns[selected_cols_idx].tolist()
print(f'Selected by Lasso: {selected_cols_idx.shape[0]} from {model.coef_.shape[0]}')
print(selected_cols)

Selected by Lasso: 18 from 177
['ask_flatten_std_5', 'ask_flatten_skew_5', 'ask_flatten_kurtosis_5', 'bid_flatten_iqr_5', 'bid_flatten_skew_5', 'bid_flatten_kurtosis_5', 'flatten_spread_5_mean', 'ask_flatten_std_10', 'bid_flatten_iqr_10', 'ask_flatten_std_15', 'bid_flatten_iqr_15', 'ask_flatten_std_25', 'bid_flatten_iqr_25', 'bid_flatten_kurtosis_25', 'bid_flatten_iqr_50', 'ask_flatten_std_100', 'bid_flatten_iqr_100', 'askSize1']


In [16]:
# from sklearn.linear_model import Lasso

# model = Lasso(alpha=0.02)
# model.fit(train_data_norm, train_target)
# predicted = model.predict(test_data_norm)
# corr_score = np.corrcoef(predicted, test_target)[0,1]
# print(f'Correlation: {corr_score:.6f}')

Correlation: 0.141188


In [21]:
from lightgbm import LGBMRegressor
model = LGBMRegressor(n_jobs=-1, num_leaves=13, learning_rate=0.01, n_estimators=300)
model.fit(train_data[selected_cols2], train_target)
predicted = model.predict(test_data[selected_cols2])
corr_score = np.corrcoef(predicted, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045451 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4632
[LightGBM] [Info] Number of data points in the train set: 2622078, number of used features: 26
[LightGBM] [Info] Start training from score 0.001149
Correlation: 0.142183


In [24]:
from lightgbm import LGBMRegressor
# model = LGBMRegressor(n_jobs=-1, num_leaves=32, learning_rate=0.01, n_estimators=300, reg_lambda=1, colsample_bytree=0.3, subsample=0.2)
model = LGBMRegressor(n_jobs=-1, num_leaves=13, learning_rate=0.01, n_estimators=300, reg_lambda=1, colsample_bytree=0.7, subsample=0.7)
model.fit(train_data[selected_cols], train_target)
predicted = model.predict(test_data[selected_cols])
corr_score = np.corrcoef(predicted, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

NameError: name 'selected_cols' is not defined

In [107]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=100)
model.fit(train_data_norm, train_target)
predicted2 = model.predict(test_data_norm)
corr_score = np.corrcoef(predicted2, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

Correlation: 0.140007


In [110]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=100)
model.fit(train_data_norm[selected_cols], train_target)
predicted3 = model.predict(test_data_norm[selected_cols])
corr_score = np.corrcoef(predicted3, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

Correlation: 0.149400


In [117]:
corr_score = np.corrcoef(predicted3 * 0.25 + predicted2 * 0.25 + predicted * 0.5, test_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

Correlation: 0.155916


In [130]:
from sklearn.model_selection import KFold, cross_val_predict
model = Ridge(alpha=1)
cross_predicted = cross_val_predict(model, X=train_data[selected_cols], y=target, cv=5)
corr_score = np.corrcoef(cross_predicted, train_target)[0,1]
print(f'Correlation: {corr_score:.6f}')

ValueError: Found input variables with inconsistent numbers of samples: [2622818, 3497666]

In [74]:
# expected_distr = (train_target.value_counts().sort_index() / (train_target.shape[0]) * test_target.shape[0]).round(0)
# expected_test_values = []
# for elem, values in expected_distr.items():
#     expected_test_values.extend([elem] * int(values))
# expected_test_values = np.array(expected_test_values)

# from scipy.stats import rankdata
# ranks = rankdata(predicted, method='min')
# rank_pred = expected_test_values[ranks]
# corr_score = np.corrcoef(rank_pred, test_target)[0,1]
# print(f'Correlation: {corr_score:.6f}')

# corr_score = np.corrcoef(rank_pred + predicted, test_target)[0,1]
# print(f'Correlation: {corr_score:.6f}')
